# Recipe 20: Build a Reusable Search Function

Create a flexible, reusable function for common search patterns.

In [ ]:
import pds.peppi as pep
from datetime import datetime
from typing import Optional

In [ ]:
def search_planetary_data(
    target: str,
    spacecraft: Optional[str] = None,
    start_date: Optional[datetime] = None,
    end_date: Optional[datetime] = None,
    processing_level: Optional[str] = None,
    max_results: Optional[int] = 100
):
    """
    Flexible search function for planetary data.
    
    Args:
        target: Name of planetary body (e.g., "Mars", "Jupiter")
        spacecraft: Optional spacecraft/rover name
        start_date: Optional start date for temporal filter
        end_date: Optional end date for temporal filter
        processing_level: Optional processing level filter
        max_results: Maximum number of results to return
    
    Returns:
        pandas DataFrame with results
    """
    client = pep.PDSRegistryClient()
    context = pep.Context()
    
    # Start building query
    query = pep.Products(client).has_target(target)
    
    # Add optional filters
    if spacecraft:
        host = context.INSTRUMENT_HOSTS.search(spacecraft)[0]
        query = query.has_instrument_host(host.lid)
    
    if start_date:
        query = query.after(start_date)
    
    if end_date:
        query = query.before(end_date)
    
    if processing_level:
        query = query.has_processing_level(processing_level)
    
    # Get observational products
    query = query.observationals()
    
    # Return as DataFrame
    return query.as_dataframe(max_rows=max_results)

## Example Usage

### Simple Query

In [ ]:
# Simple query
df = search_planetary_data(target="Mars", max_results=10)
print(f"Found {len(df)} products")
df.head()

### Complex Query with Multiple Filters

In [ ]:
# Complex query
df = search_planetary_data(
    target="Mars",
    spacecraft="curiosity",
    start_date=datetime(2020, 1, 1),
    processing_level="calibrated",
    max_results=20
)

if df is not None:
    print(f"Found {len(df)} products")
    print(df.head())

## Key Takeaways

- Reusable functions reduce code duplication
- Optional parameters provide flexibility
- Type hints improve code clarity
- Can be extended with additional filters as needed